In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

# Paths
DATASET_DIR = "dataset"
IMG_SIZE = 224
BATCH_SIZE = 32

# Map classes to days left
label_map = {
    "freshapples": 5,
    "freshbanana": 5,
    "freshoranges": 5,
    "rottenapples": 0,
    "rottenbanana": 0,
    "rottenoranges": 0
}

# Preprocessing
datagen = ImageDataGenerator(rescale=1./255)

# Base generators (keep .n available)
train_base = datagen.flow_from_directory(
    os.path.join(DATASET_DIR, "train"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="sparse",
    shuffle=True
)

test_base = datagen.flow_from_directory(
    os.path.join(DATASET_DIR, "test"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="sparse",
    shuffle=True
)

# Regression wrapper
def regression_generator(base_gen):
    while True:
        x, y = next(base_gen)
        y_reg = np.array([label_map[list(base_gen.class_indices.keys())[int(lbl)]] for lbl in y])
        yield x, y_reg

train_gen = regression_generator(train_base)
test_gen  = regression_generator(test_base)

# Steps per epoch (use .n from base generator)
train_steps = int(np.ceil(train_base.n / BATCH_SIZE))
test_steps  = int(np.ceil(test_base.n / BATCH_SIZE))

# Transfer Learning Model
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="linear")  # regression output
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Train
history = model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=test_gen,
    validation_steps=test_steps,
    epochs=15
)

# Save model
model.save("fruit_freshness.h5")
print("✅ Model training complete and saved as fruit_freshness.h5")


Found 10901 images belonging to 6 classes.
Found 2698 images belonging to 6 classes.
Epoch 1/15
341/341 ━━━━━━━━━━━━━━━━━━━━ 696s 2s/step - loss: 1.2330 - mae: 0.7951 - val_loss: 0.6534 - val_mae: 0.5714
Epoch 2/15
341/341 ━━━━━━━━━━━━━━━━━━━━ 526s 2s/step - loss: 0.5427 - mae: 0.5150 - val_loss: 0.4884 - val_mae: 0.4767
Epoch 3/15
341/341 ━━━━━━━━━━━━━━━━━━━━ 3746s 11s/step - loss: 0.4116 - mae: 0.4420 - val_loss: 0.4730 - val_mae: 0.4662
Epoch 4/15
341/341 ━━━━━━━━━━━━━━━━━━━━ 479s 1s/step - loss: 0.2968 - mae: 0.3799 - val_loss: 0.3398 - val_mae: 0.3917
Epoch 5/15
341/341 ━━━━━━━━━━━━━━━━━━━━ 513s 2s/step - loss: 0.2363 - mae: 0.3405 - val_loss: 0.3469 - val_mae: 0.3953
Epoch 6/15
341/341 ━━━━━━━━━━━━━━━━━━━━ 973s 3s/step - loss: 0.2086 - mae: 0.3185 - val_loss: 0.2828 - val_mae: 0.3500
Epoch 7/15
341/341 ━━━━━━━━━━━━━━━━━━━━ 1123s 3s/step - loss: 0.1672 - mae: 0.2878 - val_loss: 0.2758 - val_mae: 0.3554
Epoch 8/15
341/341 ━━━━━━━━━━━━━━━━━━━━ 605s 2s/step - loss: 0.1394 - mae: 0.26

✅ Model training complete and saved as fruit_freshness.h5
